# Evaluating a NeMo checkpoint on an arbitrary task

This notebook demonstrates how to extend the evaluation capabilities within the NeMo Framework container.
It guides you through the advanced configuration of an evaluation job.

For an introduction to evaluation with NVIDIA Evals Factory and the NeMo Framework, please refer to the tutorial ["Evaluating a NeMo checkpoint with lm-eval"](mmlu.ipynb) first.

In this tutorial, we will evaluate an LLM on the [WikiText-2](https://arxiv.org/abs/1609.07843) benchmark, which is available in the [NVIDIA Evals Factory lm-eval](https://pypi.org/project/nvidia-lm-eval/) package.
The evaluation utilizes the log-probabilities of the context tokens to assess how likely the input text is, according to the model.

> Note: It is recommended to run this notebook within a [NeMo Framework container](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/nemo), as it includes all necessary dependencies.

In [ ]:
import json
import signal
import subprocess

from nemo.collections.llm import api
from nemo.collections.llm.evaluation.api import EvaluationConfig, EvaluationTarget
from nemo.collections.llm.evaluation.base import list_available_evaluations
from nemo.utils import logging

logging.setLevel(logging.INFO)

## 1. Deploying the model

We will start from deploying the model.

First, you need to prepare a NeMo 2 checkpoint of the model you would like to evaluate.
For the purpose of this tutorial, we will use the Llama 3.2 1B Instruct checkpoint, which you can download from the [NGC Catalog](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/llama-3_2-1b-instruct).
Ensure that you mount the directory containing the checkpoint when starting the container.
In this tutorial, we assume that the checkpoint is accessible under the path `"/checkpoints/llama-3_2-1b-instruct_v2.0"`.

> Note: You can learn more about deployment and available server endpoints from the ["Evaluating a NeMo checkpoint with lm-eval"](mmlu.ipynb) tutorial. 

In [ ]:
# modify this variable to point to your checkpoint
# this notebook uses https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/llama-3_2-1b-instruct
CHECKPOINT_PATH = "/checkpoints/llama-3_2-1b-instruct_v2.0"

# if you are not using NeMo FW container, modify this path to point to scripts directory
SCRIPTS_PATH = "/opt/NeMo/scripts"

# modify this path if you would like to save results in a different directory
WORKSPACE = "/workspace"

In [ ]:
deploy_script = f"{SCRIPTS_PATH}/deploy/nlp/deploy_in_fw_oai_server_eval.py"

In [ ]:
deploy_process = subprocess.Popen(
    ["python", deploy_script, "--nemo_checkpoint", CHECKPOINT_PATH], 
)

In [ ]:
base_url = "http://0.0.0.0:8886"
model_name = "triton_model"

completions_url = f"{base_url}/v1/completions/"
chat_url = f"{base_url}/v1/chat/completions/"

## 2. Defining a custom evaluation workflow

NVIDIA Evals Factory packages include pre-defined evaluation configurations.
These configurations represent some of the most commonly used evaluation settings and simplify running the most frequently used benchmarks.

They can be listed using the `list_available_evaluations` function.

In [ ]:
list_available_evaluations()

However, users are not limited to this short list of benchmarks.
If you would like to evaluate a model on a different task from the underlying evaluation harness, you simply need to specify the full configuration.

For this tutorial, we will use the `wikitext` task from `lm-evaluation-harness`.
Note that for tasks without a predefined configuration you must specify the type in the `"<evaluation harness>.<task name>"` format.

Since this task uses the log-likelihoods of the input texts, we need to specify parameters for loading the tokenizer: `"tokenizer_backend"` and `"tokenizer"`.
For the model used in this example these are `"huggingface"` and `"/checkpoints/llama-3_2-1b-instruct_v2.0/context/nemo_tokenizer"`, respectively.

In [ ]:
target_config = EvaluationTarget(api_endpoint={"url": completions_url, "type": "completions"})
eval_config = EvaluationConfig(
    type="lm-evaluation-harness.wikitext",
    params={"extra": {
                "tokenizer_backend": "huggingface",
                "tokenizer": f"{CHECKPOINT_PATH}/context/nemo_tokenizer"},
           },
    output_dir=f"{WORKSPACE}/wikitext_results",
)

results = api.evaluate(target_cfg=target_config, eval_cfg=eval_config)

Finally, we can shut the model server down and inspect evaluation results.

In [ ]:
deploy_process.send_signal(signal.SIGINT)

In [ ]:
print(json.dumps(results['tasks'], indent=4))